In [ ]:
%matplotlib inline
import sys
sys.path.append("..")
from importlib import reload
import numpy as np
from matplotlib import pyplot as plt
import igraph

import pythd

In [ ]:
# Create and show two circles with some noise
dataset = (pythd.datagen.DatasetGenerator()
                .circle(center=[-4.0, 0.0], radius=4.0, noise=0.1, num_points=200)
                .circle(center=[4.0, 0.0], radius=4.0, noise=0.1, num_points=200)).get()
plt.plot(dataset[:, 0], dataset[:, 1], ".")
plt.show()

In [ ]:
# Setup MAPPER
reload(pythd)
reload(pythd.filter)
reload(pythd.cover)
filt = pythd.filter.IdentityFilter()
f_x = filt(dataset)
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 5, 0.5)
clustering = pythd.clustering.HierarchicalClustering()

In [ ]:
# Run MAPPER
reload(pythd)
reload(pythd.mapper)
mapper = pythd.mapper.MAPPER(filter=filt, cover=cover, clustering=clustering)
res = mapper.run(dataset, f_x=f_x)

In [ ]:
# Create igraph graph
res.compute_k_skeleton(k=1)
g = res.get_igraph_network()
print(g)

In [ ]:
# Visualize the graph
layout = g.layout_kamada_kawai()
igraph.plot(g, layout=layout)

In [ ]:
# Visualize with networkx
import networkx as nx
g = res.get_networkx_network()
nx.draw(g)

In [ ]:
from importlib import reload
import pythd.plotting as plot
reload(plot)
plot.draw_2_skeleton(res.compute_k_skeleton(k=2), layout)